In [6]:
import numpy as np;
import math;
from enum import Enum;
import time;

In [7]:
class Edge:
    
    def fields(self):
        self.vertexA = None;
        self.vertexB = None;
        self.distance = -1;
    
    def __init__(self, vertexA, vertexB, distance):
        self.fields(); 
        self.vertexA = vertexA;
        self.vertexB = vertexB;
        self.distance = distance;
        
    def isTheSame(self, _vertexA, _vertexB):
        return ((_vertexA == self.vertexA or 
                _vertexA == self.vertexB) and 
                (_vertexB == self.vertexA or 
                 _vertexB == self.vertexB));

In [8]:
class Vertex:
    
    count = 0;
    
    def fields(self):
        self.ID = -1;
        self.label = -1;
        self.directions = [];
        self.graph = None;
    
    def __init__(self, ID, graph):
        self.fields();
        self.ID = int(ID);
        self.graph = graph;
        
    def addDirection(self, d):
        self.directions.append(d);
    
    def search(self, endPoint):
        Vertex.count += 1;
        #print("Distance to " + str(self.ID) + " : \t" + str(self.label));
        
        #self.graph.printLabeledVertices();
        
        if(self == endPoint):
            print("Got to the end! With path length: " + str(self.label));
            return;
        if( endPoint.label != -1 and self.label > endPoint.label):
            #print("Not neccesary to take path longer, than alredy exist! " + str(self.ID));
            return;
        
        if(len(self.directions) == 0):
            #print("Dead End! at [" + str(self.ID) + "]");
            pass;
        else:
            for d in self.directions:      
                #calc distance to be
                edgeLen = self.graph.getDistanceBetween(self, d);
                dist = self.label + edgeLen;
                dist = round(dist, 4);
                if(d.label > dist or d.label == -1):
                    #print("Now in " + str(self.ID) + " going into " + str(d.ID));
                    #print("Path to destination " + str(d.label) + " Distance " + str(dist));
                    d.label = dist;
                    d.search(endPoint);
                if(d.label+edgeLen < self.label):
                    #print("Shortent the path from " + str(self.label) + " to: " +str(d.label+edgeLen) + " ID: "+ str(self.ID));
                    self.label = d.label+edgeLen;
                
    def returnTo(self, startVertex):
        self.graph.path.append(self);
        if(startVertex == self):
            #print("Got back!");
            return;
        smallestDist = math.inf;
        parentVertex = None;
        for d in self.directions:
            if(d.label != -1 and d.label < smallestDist):
                smallestDist = d.label;
                parentVertex = d;
        parentVertex.returnTo(startVertex);
        
        
    def rearrangeDestinations(self):
        self.directions.sort(key=lambda x: self.graph.getDistanceBetween(self, x), reverse=True);

In [9]:
class Graph:
    
    def fields(self):
        self.vertices = [];
        self.edges = [];
        self.path = [];
    
    
    def __init__(self, filename):
        self.fields();
        matrix = Graph.load_graph(filename);
        
        for i in range(len(matrix)):
            self.createEdge(matrix[i,0], matrix[i,1], matrix[i,2]);
        for v in self.vertices:
            v.rearrangeDestinations();
    
    @staticmethod
    def load_graph(path):
        edges = np.array([]);
        with open(path, 'r', encoding='utf-8', errors='ignore') as g_file:
            next(g_file); # skip the header line

            for line in g_file:
                try:
                    fields = line.split(",");
                    edges = np.append(edges, [int(fields[0]), int(fields[1]), float(fields[2])], axis=None);
                    edges = np.reshape(edges, (-1,3))
                except Exception as e:
                    pass
        return edges;
    
    def createEdge(self, vertexA, vertexB, distance):
        #if vertex with id vertexA doesn't exist -> create
        #else get vertex with id vertexA
        va = self.getVertex(vertexA);
        if(va == None):
            va = Vertex(vertexA, self);
            self.vertices.append(va);
        
        #if vertex with id vertexB doesn't exist -> create
        #else get vertex with id vertexB
        vb = self.getVertex(vertexB);
        if(vb == None):
            vb = Vertex(vertexB, self);
            self.vertices.append(vb);
        
        #Add vb as a direction to va
        va.addDirection(vb);
        
        #create an edge and add to edges array
        self.edges.append(Edge(va, vb, distance));
        
    def getVertex(self, ID):
        for v in self.vertices:
            if( v.ID == ID):
                return v;
        return None;
    
    def getDistanceBetween(self, vertexA, vertexB):
        for edge in self.edges:
            if(edge.isTheSame(vertexA, vertexB)):
                #print("Distance between " + str(vertexA.ID) + " and " + str(vertexB.ID) + ": " + str(edge.distance));
                return edge.distance;
        print("Edge was not found!!!");
        return 0;
        
    def startSearchAt(self, startID, endID):
        self.path = [];
        
        startPoint = self.getVertex(startID);
        endPoint = self.getVertex(endID);
        
        startPoint.label = 0;
        startPoint.search(endPoint);
        
        endPoint.returnTo(startPoint);
        self.path.reverse();
        
        
    def printLabeledVertices(self, width=10, height=10):
        
        spacing = 2;
        places = 5
        decimal = 2;
        strFormat = "%" + str(places) + "." + str(decimal) + "f";
        
        print("_"*(width*(places+spacing)+3))
        for i in range(width):
            print("|", end = " ");
            for j in range(height):
                v = self.getVertex(i*width + j);
                if(v == None):
                    print("■"*places, end = " "*spacing);
                else: 
                    if(v.label == -1):
                        print("□"*places, end = " "*spacing);
                    else:
                        print(strFormat %v.label, end = " "*spacing);
            print("|", end = "");
            print();
        print("‾"*(width*(places+spacing)+3))

In [10]:
g = Graph("FullMaze.txt");
Vertex.count = 0;
start_time = time.time()
g.startSearchAt(81, 16);
elapsed_time = time.time() - start_time
print("Time took: " + str(elapsed_time) + " s?");
print("Total iterations: " + str(Vertex.count));
print("Path: ");

for d in g.path:
    if(d.ID != 16):
        print(d.ID, end=" -> ");
    else:
        print(d.ID, end="");
print();
for i in range(10):
    for j in range(10):
        v = g.getVertex(i*10 + j);
        if(v == None):
            print("----", end = "\t");
        else:     
            print("%5.2f" %v.label, end = "\t");
    print();

Got to the end! With path length: 23.15
Got to the end! With path length: 22.33
Got to the end! With path length: 21.74
Got to the end! With path length: 20.33
Got to the end! With path length: 19.51
Got to the end! With path length: 18.1
Got to the end! With path length: 16.69
Got to the end! With path length: 15.28
Got to the end! With path length: 13.87
Time took: 0.0807807445526123 s?
Total iterations: 340
Path: 
81 -> 92 -> 83 -> 73 -> 64 -> 65 -> 66 -> 57 -> 48 -> 38 -> 27 -> 16
10.05	 9.64	10.05	10.46	10.87	----	14.87	14.46	14.05	14.46	
 9.05	 8.64	 9.05	 9.46	 9.87	----	13.87	13.46	13.05	13.46	
 8.05	 7.64	 8.05	 8.46	 9.23	----	12.87	12.46	12.05	12.46	
 6.23	 6.64	 7.05	 7.82	 8.23	----	----	11.46	11.05	11.46	
 5.23	 5.64	 6.64	 6.82	 7.23	 7.64	----	----	10.05	11.05	
 3.41	----	----	 5.82	 6.23	 6.64	 7.64	 8.64	 9.64	10.64	
 2.41	 2.00	----	 4.82	 5.23	 6.23	 7.23	 8.23	 9.23	10.23	
 1.41	 1.00	----	 3.82	----	 6.64	 7.64	 8.64	 9.64	10.64	
 1.00	 0.00	----	 2.82	----	 7.64	